In [99]:
from selenium import webdriver
import pandas as pd

In [100]:
# target url
url = "http://corners.gmarket.co.kr/Bestsellers"

In [101]:
driver = webdriver.Chrome()

In [102]:
driver.get(url)

In [103]:
# specify "ul" under id=gBestWrap, class=best-list
bestlist = driver.find_elements_by_css_selector("#gBestWrap .best-list")[1]
bestlist.text

'1\n쁘띠뮤/신학기 아동정장 원피스 레깅스 트레이닝\n29,000원\n8,900원 69%\n2\n모드니9 아동복 인기아이템\n23,000원\n6,900원 70%\n3\n[베베숲]아기물티슈 브랜드 딜 10팩 행사\n19,900원\n11,900원 40%\n4\n봄신상 빅사이즈 트레이닝세트/원피스/팬츠\n18,900원\n5,900원 68%\n5\n[오이보스](현대백화점1관)[오이보스/EUBOS] 베이비 집중크림 100ml+선택2종+정품 증정\n50,260원\n45,250원 9%\n6\n9900원~ 여성단화 슬립온 단화 스니커즈 신발 운동화\n33,000원\n9,900원 70%\n7\n((20%쿠폰))클리오/페리페라外 2월 빅세일(~2/25)\n20,000원\n14,000원 30%\n8\n듀폰소재로 만든 나우썸 경량패딩(nowsum light jumper)\n39,900원\n19,950원 50%\n9\n[아워홈]게살/돼지고기/칠리/새우/치킨/불고기볶음밥250g10봉\n25,000원\n11,900원 52%\n10\n볼륨앤소프트 천연펄프 화장지 30롤 3팩\n30,000원\n17,900원 40%\n11\n[락앤락]비스프리 물병 1+1/패턴물병 2개 /보냉병 텀블러 보틀\n23,000원\n6,900원 70%\n12\n루케테 신상 스니커즈/운동화/벨크로/신발/단화/남성\n66,300원\n19,900원 69%\n13\n스위티아 봄신상 롱티셔츠/루즈핏/원피스/후드티\n32,900원\n9,900원 69%\n14\n[웅진]빅토리아 플레인 탄산수/탄산음료 500mlx20 6종택1\n15,000원\n7,900원 47%\n15\n[섀르반]섀르반 아동 상의/하의모음 티셔츠/니트/원피스\n65,000원\n19,800원 69%\n16\n블론디 44~88 여자기모롱티/레이어드 치마/빅사이즈\n25,900원\n7,900원 69%\n17\n[군기저귀]군기저귀 한정판 오리지날 팬티 3팩 / 밴드3팩\n49,900원\n29,900원 40%\n18\n[GFresh]지리산 구례 고로쇠수액 4.5L 1병

In [104]:
# returns all elements under <li> 
# we need to get all of them through iteration
items = bestlist.find_elements_by_css_selector("li")
items

[<selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", element="0.7254729244945799-10")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d42c15f7b7c1c7c123f096c1c1d1ba7c", e

In [105]:
# note that the ".s-price" has two text elements, and we need to split it.
print(item.find_element_by_css_selector(".item_price .s-price").text)
print(item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0])
print(item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[1])

16,900원 32%
16,900원
32%


In [106]:
# create DataFrame to input data
df = pd.DataFrame(columns=["item_name", "original_price", "current_price", "image_link", "item_link"])
df

,item_name,original_price,current_price,image_link,item_link


In [107]:
# create function to leave only integers when scraping price data
def to_int(price):
    return int(price.replace(",","").replace("원",""))

In [108]:
for item in items:
    idx = item.find_element_by_css_selector("p").text
    img_link = item.find_element_by_css_selector("img").get_attribute("src")
    item_link = item.find_element_by_css_selector(".itemname").get_attribute("href")
    name = item.find_element_by_css_selector(".itemname").text
    o_price = item.find_element_by_css_selector(".item_price .o-price").text
    s_price = item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0]
    o_price = o_price if o_price else s_price # if no original price, then replace with selling price
    
    data = {"item_name" : name, 
            "original_price" : to_int(o_price),
            "current_price" : to_int(s_price),
            "image_link" : img_link,
            "item_link": item_link, }
    
    # input scrapped data into pandas DataFrame
    df.loc[len(df)] = data

In [109]:
# note that the ".s-price" has two text elements, and we had to split it.
print(item.find_element_by_css_selector(".item_price .s-price").text)
print(item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0])
print(item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[1])

20,900원 51%
20,900원
51%


In [110]:
# return DataFrame table
df.head()

,item_name,original_price,current_price,image_link,item_link
0,쁘띠뮤/신학기 아동정장 원피스 레깅스 트레이닝,29000,8900,http://gdimg.gmarket.co.kr/946176130/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
1,모드니9 아동복 인기아이템,23000,6900,http://gdimg.gmarket.co.kr/1266609567/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
2,[베베숲]아기물티슈 브랜드 딜 10팩 행사,19900,11900,http://gdimg.gmarket.co.kr/804104212/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
3,봄신상 빅사이즈 트레이닝세트/원피스/팬츠,18900,5900,http://gdimg.gmarket.co.kr/899311960/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
4,[오이보스](현대백화점1관)[오이보스/EUBOS] 베이비 집중크림 100ml+선택2...,50260,45250,http://gdimg.gmarket.co.kr/1267434080/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...


In [112]:
# calculate discount rate 
df['dis_rate'] = 100 - (df['current_price']/df['original_price'] * 100)
df['dis_rate'] = df['dis_rate'].astype(dtype="float64")
df['dis_rate'] = round(df['dis_rate'], 2)
df = df.sort_values('dis_rate', ascending=False).reset_index().rename(columns={"index":"rank"})
df

,rank,item_name,original_price,current_price,image_link,item_link,dis_rate
0,112,[아망떼]순면100%모닝벨 매트리스커버/이불커버/침대패드,33000,8900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,73.03
1,190,봄신상 원피스 레깅스 기모최종가 슈퍼특가 아동복,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
2,66,삠뽀요/봄신상 원피스9900원 베이비 우주복 돌복,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,1,모드니9 아동복 인기아이템,23000,6900,http://gdimg.gmarket.co.kr/1266609567/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,110,[머스트비]케네스콜/YK038/바닐라비 단독기획 트렌치/자켓/야상,133000,39900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
5,5,9900원~ 여성단화 슬립온 단화 스니커즈 신발 운동화,33000,9900,http://gdimg.gmarket.co.kr/838690913/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
6,196,이사도라 사랑스러운 그녀 플랫/로퍼/단화,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
7,58,봄구두 미들힐 하이힐 슬링백 로퍼 메리제인 스틸레토,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
8,117,[디즈니]디즈니 아동 캐릭터 실내화 슬리퍼 학생 덧신 샌들,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
9,185,[젤리스푼]아동복/봄신상오픈/상하복/티셔츠/레깅스/팬츠/점퍼,43000,12900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00


In [113]:
df.to_csv("gMarket_best_item", encoding="utf-8")

In [114]:
data = pd.read_csv("gMarket_best_item", index_col=0)

In [115]:
data

,rank,item_name,original_price,current_price,image_link,item_link,dis_rate
0,112,[아망떼]순면100%모닝벨 매트리스커버/이불커버/침대패드,33000,8900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,73.03
1,190,봄신상 원피스 레깅스 기모최종가 슈퍼특가 아동복,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
2,66,삠뽀요/봄신상 원피스9900원 베이비 우주복 돌복,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,1,모드니9 아동복 인기아이템,23000,6900,http://gdimg.gmarket.co.kr/1266609567/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,110,[머스트비]케네스콜/YK038/바닐라비 단독기획 트렌치/자켓/야상,133000,39900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
5,5,9900원~ 여성단화 슬립온 단화 스니커즈 신발 운동화,33000,9900,http://gdimg.gmarket.co.kr/838690913/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
6,196,이사도라 사랑스러운 그녀 플랫/로퍼/단화,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
7,58,봄구두 미들힐 하이힐 슬링백 로퍼 메리제인 스틸레토,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
8,117,[디즈니]디즈니 아동 캐릭터 실내화 슬리퍼 학생 덧신 샌들,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
9,185,[젤리스푼]아동복/봄신상오픈/상하복/티셔츠/레깅스/팬츠/점퍼,43000,12900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
